In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
raw_data = pd.read_csv('./data/raw_data.csv')
raw_data.head()

,raw_tweet_data
0,@chardinghh In meiner Branche sind wir damit g...
1,@crittersport 🤣
2,Ich vermisse beim Stundeneintragen den Punkt „...
3,@lawen4cer N‘Trotzki.
4,@ktschk ER HAT UNS DIE FREUDE GESCHENKT!


## Some ideas:
- `@` + word + ` ` 
- Lowercase everything

### Remove
- Numbers
- Hashtags
- Links
- Special characters such as *:, (, ), etc.*
- Emojis
- `"RT"`

### Special characters
- \ - " "( )  / : _ * $ + 

In [3]:
data = raw_data['raw_tweet_data'].copy()

In [4]:
data

0       @chardinghh In meiner Branche sind wir damit g...
1                                         @crittersport 🤣
2       Ich vermisse beim Stundeneintragen den Punkt „...
3                                   @lawen4cer N‘Trotzki.
4                @ktschk ER HAT UNS DIE FREUDE GESCHENKT!
                              ...                        
1991                                           @andijah 😇
1992    @andijah Ganz spontan: Guru oder Crack – ist a...
1993    RT @andijah: Kurze Zwischenfrage ins virtuelle...
1994              RT @BlakesWort: https://t.co/p0RpcvNb6j
1995    @dpd_de Done. Die Sendungsverfolgung zeigt imm...
Name: raw_tweet_data, Length: 1996, dtype: object

In [6]:
# Hashtag filter
no_hashtag = data.apply(lambda x: re.sub('#[\w]+','', x)) 

# Mention filter
no_mention = no_hashtag.apply(lambda x: re.sub("@([A-Za-z0-9_ßäöüÄÖÜÇâêîôûéàèùëïç]+)|([^0-9A-Za-z_ßäöüÄÖÜâêîôûéàèùëïç \t])|(\w+:\/\/\S+)",' ', x))

# Remove RT
no_rt = no_mention.apply(lambda x: re.sub('RT','', x))

# Remove numbers
no_num = no_rt.apply(lambda x: re.sub('[0-9]+','', x))

# Remove extra underscores
no_underscore = no_num.apply(lambda x: re.sub('_','', x))

# Remove leading and trailing whitespaces
no_space = no_underscore.apply(lambda x: x.strip())

# Lowercase the words
lowercase = no_space.apply(lambda x: x.lower())

# Remove extra spaces in between words
cleaned = lowercase.apply(lambda x: " ".join(x.split()))


In [7]:
# Save to dataframe
raw_data['clean_data'] = cleaned

In [8]:
# Remove any empty rows
empty_rows = raw_data[raw_data.clean_data == ''].index
empty_removed = raw_data.drop(empty_rows, axis=0)
empty_removed.reset_index(drop=True, inplace=True)

empty_removed

,raw_tweet_data,clean_data
0,@chardinghh In meiner Branche sind wir damit g...,in meiner branche sind wir damit ganz weit vorn
1,Ich vermisse beim Stundeneintragen den Punkt „...,ich vermisse beim stundeneintragen den punkt e...
2,@lawen4cer N‘Trotzki.,n trotzki
3,@ktschk ER HAT UNS DIE FREUDE GESCHENKT!,er hat uns die freude geschenkt
4,@tantekatya @HeikoMaas @ZDFheute Das ist die i...,das ist die innere kälte dass er nicht mehr am...
...,...,...
1867,Merke: „x-mal so viel“ ist nicht dasselbe wie ...,merke x mal so viel ist nicht dasselbe wie x m...
1868,@nikahle @andijah Profi finde ich super!,profi finde ich super
1869,@andijah Ganz spontan: Guru oder Crack – ist a...,ganz spontan guru oder crack ist aber beides a...
1870,RT @andijah: Kurze Zwischenfrage ins virtuelle...,kurze zwischenfrage ins virtuelle großraumbüro...


In [9]:
empty_removed.isna().sum()

raw_tweet_data    0
clean_data        0
dtype: int64

In [10]:
empty_removed.to_csv('./data/cleaned_data.csv', index=False)